# Imports

In [ ]:
from enum import Enum
from os import listdir
import os.path

In [ ]:
from tensorflow.keras.applications import vgg16
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.data import Dataset
import numpy as np

In [ ]:
# Let keras load images that might be too small
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Constants

In [ ]:
LOG_SEVERITY = 1

IMG_TGT_SZ = (224, 224)
IMGS_BASE_PATH = os.path.join('..', 'datasets', 'COVID-19 Dataset', 'CT', 'COVID')
#IMGS_BASE_PATH = os.path.join('..', 'datasets', 'ieee8023')

# Support Functions

In [ ]:
class Severity(Enum):
    DEBUG = 1
    INFO = 2
    WARNING = 3

def log_msg(msg, severity=Severity.INFO):
    if LOG_SEVERITY <= severity.value:
        print(f'{severity.name}: {msg}')

In [ ]:
def preprocess_images(images, model_preprocess=None):
    "Preprocess a list of image files to be used with a CNN 224x224 model"
    pil_images = [load_img(image, target_size=IMG_TGT_SZ) for image in images]
    log_msg(f'Loaded {len(images)} images.')
    np_images = np.array([img_to_array(image) for image in pil_images])
    log_msg(f'Converted to numpy arrays with shape {np_images.shape}.')
    if model_preprocess:
        np_images = model_preprocess(np_images)
    return np_images

# Primary Execution flow

In [ ]:
files = listdir(IMGS_BASE_PATH)

images = [os.path.join(IMGS_BASE_PATH, f) for f in files 
          if f.endswith(('png', 'PNG', 'jpg', 'JPG', 'jpeg', 'JPEG'))]
image_names = [os.path.split(f)[1].rpartition('.')[0] for f in images]

start = perf_counter()

log_msg(f'Loading {len(image_names)} from {IMGS_BASE_PATH}')
log_msg(f'Some example images: {images[:5]}', Severity.DEBUG)
np_images = preprocess_images(images, vgg16.preprocess_input)

end = perf_counter()
log_msg(f'Loading images took {end-start} seconds.', Severity.DEBUG)

tfdata = Dataset.from_tensor_slices(np_images)

In [ ]:
ct_covid = tfdata

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
gridx, gridy = 3, 4
rand_imgs = np.random.randint(len(images), size=(gridx*gridy))
rand_files = [images[ndx] for ndx in rand_imgs]
plt.figure(figsize=(11, 15))
for ndx, img in enumerate(rand_files, 1):
    plt.subplot(gridx, gridy, ndx)
    plt.imshow(load_img(img))

In [ ]:
plt.figure(figsize=(11, 15))
for ndx, img in enumerate(rand_files, 1):
    plt.subplot(gridx, gridy, ndx)
    plt.imshow(load_img(img, target_size=IMG_TGT_SZ))

In [ ]:
# Load model with only cnn-layers
model_vgg16_conv = vgg16.VGG16(weights='imagenet', include_top=False)
input_layer = Input(shape=(*IMG_TGT_SZ, 3), name = 'image_input')
output_vgg16_conv = model_vgg16_conv(input_layer)

In [ ]:
#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(2, activation='softmax', name='predictions')(x)

#Create your own model 
my_model = Model(inputs=input_layer, outputs=x)

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model.summary()

# Predict images w/ vgg16

In [ ]:
model = vgg16.VGG16()
yhat = model.predict(np_images)
predictions = vgg16.decode_predictions(yhat)

In [ ]:
print('Top Predictions by vgg16')
print('-'*40)
for ndx, name in enumerate(image_names):
    print(f'{name}: {predictions[ndx][0][1]}')